### Spiking Neural Network Builder example using NeuroWorkflow

In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# import nest and reset the kernel (just in case it might be needed)
import nest
nest.ResetKernel()


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.7.0
 Built: Mar  4 2025 17:27:39

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [3]:
# Add path to NeuroWorflow
# Add the src directory to the Python path if needed
src_path = os.path.abspath(os.path.join(os.getcwd(), '../src'))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Import NeuroWorkflow components (custom nodes)
from neuroworkflow import WorkflowBuilder
from neuroworkflow.nodes.network.SNNbuilder_SingleNeuron import SNNbuilder_SingleNeuron
from neuroworkflow.nodes.network.SNNbuilder_Population import SNNbuilder_Population
from neuroworkflow.nodes.network.SNNbuilder_Connection import SNNbuilder_Connection
from neuroworkflow.nodes.stimulus.SNNbuilder_Stimulation import SNNbuilder_Stimulation
from neuroworkflow.nodes.stimulus.SNNbuilder_Recordable import SNNbuilder_Recordable
from neuroworkflow.nodes.simulation.SNNbuilder_Simulation import SNNbuilder_Simulation

### Create a Single excitatory neuron:


In [4]:

# Create an instance of the node SNNbuilder_SingleNeuron
my_neuron = SNNbuilder_SingleNeuron("pyramidal neuron")
    
# Configure the neuron with biological parameters
my_neuron.configure(
    # Biological identification
    name="Pyramidal Neuron L2/3",
    acronym="PyL23",
    model_type="point_process",
    cell_class="excitatory",
        
    # Signaling properties
    neurotransmitter_types=["AMPA", "NMDA"],
    psp_amplitudes={"AMPA": 0.8, "NMDA": 0.4},
    rise_times={"AMPA": 2.0, "NMDA": 15.0},
        
    # Morphological properties
    dendrite_extent=300.0,  # μm
    dendrite_diameter=2.5,  # μm
        
    # Activity properties
    firing_rate_resting=[2.0, 8.0],    # Hz
    firing_rate_active=[15.0, 40.0],   # Hz
    firing_rate_maximum=[60.0, 120.0], # Hz
        
    # NEST model parameters
    nest_model="iaf_psc_alpha",
    nest_parameters={
        "V_th": -50.0,      # Threshold potential (mV)
        "C_m": 200.0,       # Membrane capacitance (pF)
        "tau_m": 15.0,      # Membrane time constant (ms)
        "V_reset": -70.0,   # Reset potential (mV)
        "t_ref": 2.0,        # Refractory period (ms)
        "I_e": 200.0,        #external current (pA)
    },
        
    # Execution options
    execution_mode="both",  # Execute and generate script
    script_format="both"    # Python and notebook formats
    )
    
print(f"Neuron: {my_neuron._parameters['name']} ({my_neuron._parameters['acronym']})")
print(f"Neuron: {my_neuron._parameters}")

Neuron: Pyramidal Neuron L2/3 (PyL23)
Neuron: {'name': 'Pyramidal Neuron L2/3', 'acronym': 'PyL23', 'model_type': 'point_process', 'cell_class': 'excitatory', 'neurotransmitter_types': ['AMPA', 'NMDA'], 'psp_amplitudes': {'AMPA': 0.8, 'NMDA': 0.4}, 'rise_times': {'AMPA': 2.0, 'NMDA': 15.0}, 'dendrite_extent': 300.0, 'dendrite_diameter': 2.5, 'firing_rate_resting': [2.0, 8.0], 'firing_rate_active': [15.0, 40.0], 'firing_rate_maximum': [60.0, 120.0], 'firing_rate_disease': [0.1, 2.0], 'nest_model': 'iaf_psc_alpha', 'nest_parameters': {'V_th': -50.0, 'C_m': 200.0, 'tau_m': 15.0, 'V_reset': -70.0, 't_ref': 2.0, 'I_e': 200.0}, 'template_suffix': '_custom', 'execution_mode': 'both', 'script_format': 'both'}


In [5]:
#my_neuron.get_info() #uncomment to check the info of a node

In [6]:
my_neuron._parameters

{'name': 'Pyramidal Neuron L2/3',
 'acronym': 'PyL23',
 'model_type': 'point_process',
 'cell_class': 'excitatory',
 'neurotransmitter_types': ['AMPA', 'NMDA'],
 'psp_amplitudes': {'AMPA': 0.8, 'NMDA': 0.4},
 'rise_times': {'AMPA': 2.0, 'NMDA': 15.0},
 'dendrite_extent': 300.0,
 'dendrite_diameter': 2.5,
 'firing_rate_resting': [2.0, 8.0],
 'firing_rate_active': [15.0, 40.0],
 'firing_rate_maximum': [60.0, 120.0],
 'firing_rate_disease': [0.1, 2.0],
 'nest_model': 'iaf_psc_alpha',
 'nest_parameters': {'V_th': -50.0,
  'C_m': 200.0,
  'tau_m': 15.0,
  'V_reset': -70.0,
  't_ref': 2.0,
  'I_e': 200.0},
 'template_suffix': '_custom',
 'execution_mode': 'both',
 'script_format': 'both'}

### Create a population of exitatory neurons (using the previous neuron model):

In [7]:
# Create a population 
my_pop = SNNbuilder_Population('cortical_population')
    
# Configure the population parameters
my_pop.configure(
    name='Layer 2/3 Pyramidal Neurons',
    acronym='L23Pyr',
    brain_region='Primary Visual Cortex',
    population_size= 800,
    tissue_reference={ #this is a free format dictionary
        'density_per_mm3': 25000,
        'tissue_volume_mm3': 0.2,
        'reference_study': 'Schüz & Palm (1989)',
        'species': 'human',
        'estimation_method': 'stereological counting'
    },
    spatial_dimensions='3D',
    spatial_bounds={
        'x': (0.0, 2.0),
        'y': (0.0, 2.0),
        'z': (0.0, 1.0)
    },
    model_type='point_process',
    # Execution options
    execution_mode="both",  # Execute and generate script
    script_format="both"    # Python and notebook formats
    )
print(f"Population: {my_pop._parameters}")

Population: {'name': 'Layer 2/3 Pyramidal Neurons', 'acronym': 'L23Pyr', 'brain_region': 'Primary Visual Cortex', 'model_type': 'point_process', 'population_size': 800, 'tissue_reference': {'density_per_mm3': 25000, 'tissue_volume_mm3': 0.2, 'reference_study': 'Schüz & Palm (1989)', 'species': 'human', 'estimation_method': 'stereological counting'}, 'spatial_dimensions': '3D', 'spatial_bounds': {'x': (0.0, 2.0), 'y': (0.0, 2.0), 'z': (0.0, 1.0)}, 'custom_positions': None, 'mean_firing_rate': None, 'execution_mode': 'both', 'script_format': 'both'}


In [8]:
my_pop._parameters

{'name': 'Layer 2/3 Pyramidal Neurons',
 'acronym': 'L23Pyr',
 'brain_region': 'Primary Visual Cortex',
 'model_type': 'point_process',
 'population_size': 800,
 'tissue_reference': {'density_per_mm3': 25000,
  'tissue_volume_mm3': 0.2,
  'reference_study': 'Schüz & Palm (1989)',
  'species': 'human',
  'estimation_method': 'stereological counting'},
 'spatial_dimensions': '3D',
 'spatial_bounds': {'x': (0.0, 2.0), 'y': (0.0, 2.0), 'z': (0.0, 1.0)},
 'custom_positions': None,
 'mean_firing_rate': None,
 'execution_mode': 'both',
 'script_format': 'both'}

### Create a Single inhibitory neuron:


In [9]:
my_neuron_inh = SNNbuilder_SingleNeuron("Parvalbumin-expressing interneuron")
    
# Configure the neuron with biological parameters
my_neuron_inh.configure(
    # Biological identification
    name="PV interneuron",
    acronym="PV",
    model_type="point_process",
    cell_class="inhibitory",
        
    # NEST model parameters
    nest_model="iaf_psc_alpha",
    nest_parameters={
        "C_m": 200.0,       # Membrane capacitance (pF)
    },
        
    # Execution options
    execution_mode="both",  # Execute and generate script
    script_format="both"    # Python and notebook formats
    )
    
print(f"Neuron: {my_neuron_inh._parameters['name']} ({my_neuron_inh._parameters['acronym']})")
print(f"Neuron: {my_neuron_inh._parameters}")

Neuron: PV interneuron (PV)
Neuron: {'name': 'PV interneuron', 'acronym': 'PV', 'model_type': 'point_process', 'cell_class': 'inhibitory', 'neurotransmitter_types': ['AMPA', 'NMDA'], 'psp_amplitudes': {'AMPA': 0.5, 'NMDA': 0.3}, 'rise_times': {'AMPA': 2.0, 'NMDA': 10.0}, 'dendrite_extent': 200.0, 'dendrite_diameter': 2.0, 'firing_rate_resting': [1.0, 5.0], 'firing_rate_active': [10.0, 30.0], 'firing_rate_maximum': [50.0, 100.0], 'firing_rate_disease': [0.1, 2.0], 'nest_model': 'iaf_psc_alpha', 'nest_parameters': {'C_m': 200.0}, 'template_suffix': '_custom', 'execution_mode': 'both', 'script_format': 'both'}


### Create a population of inhibitory neurons (using the previous neuron model):

In [10]:
# Create a population 
my_pop_inh = SNNbuilder_Population('inhibitory cortical population')
    
# Configure the population parameters
my_pop_inh.configure(
    name='Layer of PV neurons',
    acronym='PVpop',
    brain_region='Primary Visual Cortex',
    population_size= 100,
    spatial_dimensions='3D',
    spatial_bounds={
        'x': (0.0, 2.0),
        'y': (0.0, 2.0),
        'z': (0.0, 1.0)
    },
    model_type='point_process',
    # Execution options
    execution_mode="both",  # Execute and generate script
    script_format="both"    # Python and notebook formats
    )
print(f"Population: {my_pop_inh._parameters}")

Population: {'name': 'Layer of PV neurons', 'acronym': 'PVpop', 'brain_region': 'Primary Visual Cortex', 'model_type': 'point_process', 'population_size': 100, 'tissue_reference': {}, 'spatial_dimensions': '3D', 'spatial_bounds': {'x': (0.0, 2.0), 'y': (0.0, 2.0), 'z': (0.0, 1.0)}, 'custom_positions': None, 'mean_firing_rate': None, 'execution_mode': 'both', 'script_format': 'both'}


In [11]:
my_pop_inh._parameters

{'name': 'Layer of PV neurons',
 'acronym': 'PVpop',
 'brain_region': 'Primary Visual Cortex',
 'model_type': 'point_process',
 'population_size': 100,
 'tissue_reference': {},
 'spatial_dimensions': '3D',
 'spatial_bounds': {'x': (0.0, 2.0), 'y': (0.0, 2.0), 'z': (0.0, 1.0)},
 'custom_positions': None,
 'mean_firing_rate': None,
 'execution_mode': 'both',
 'script_format': 'both'}

### Create a workflow instance and add the created nodes

In [12]:
# Create a workflow builder
workflow = WorkflowBuilder("Simple population of spiking neurons")

# Add nodes to the workflow
workflow.add_node(my_neuron)
workflow.add_node(my_neuron_inh)
workflow.add_node(my_pop)
workflow.add_node(my_pop_inh)


In [13]:
# Get port names from any node instance

my_neuron_output_ports = list(my_neuron.get_info()['output_ports'].keys())  
print(f"Neuron output ports: {my_neuron_output_ports}")

print(f"Population input ports: {my_pop.get_info()['input_ports'].keys()}")    

Neuron output ports: ['nest_neuron', 'nest_model_name', 'python_script', 'notebook_cell', 'neuron_metadata', 'biological_properties', 'nest_properties']
Population input ports: dict_keys(['nest_neuron', 'nest_model_name', 'python_script', 'notebook_cell', 'neuron_metadata', 'biological_properties', 'nest_properties'])


### Connect the nodes by input and output ports (source node name, output port, target node name, input port)

In [14]:
# Connect the nodes by ports

workflow.connect(
    my_neuron.name, my_neuron_output_ports[1], 
    my_pop.name, "nest_model_name"
)
workflow.connect(
    my_neuron.name, my_neuron_output_ports[-2], 
    my_pop.name, "biological_properties"
)
workflow.connect(
    my_neuron_inh.name, 'nest_model_name', 
    my_pop_inh.name, "nest_model_name"
)
workflow.connect(
    my_neuron_inh.name, 'biological_properties', 
    my_pop_inh.name, "biological_properties"
)

In [15]:
# count the connection number in the workflow
workflow.get_connection_count()

4

### Connect Exc.to Inh populations using NEST connection rules

In [16]:
# Create a connection
EI_connect = SNNbuilder_Connection('connection from Exc. to Inh Neurons.')
    
# Configure the connection rule parameters

EI_connect.configure(
    name='EI indegree',
    connection_rule='fixed_indegree',
    connection_dict={'allow_autapses': False}, 
    synapse_model= 'static_synapse',
    synapse_dict= {'weight': 4.0, 'delay': 1.0},

    # Execution options
    execution_mode="both",  # Execute and generate script
    script_format="both",    # Python and notebook formats

    redundancy=100.0 #to adjust the indegree
    )
print(f"Rule: {EI_connect._parameters}")

Rule: {'name': 'EI indegree', 'connection_rule': 'fixed_indegree', 'connection_dict': {'allow_autapses': False}, 'synapse_model': 'static_synapse', 'synapse_dict': {'weight': 4.0, 'delay': 1.0}, 'template_suffix': '_custom', 'axon_organization': 'focused', 'axon_radius': 0.5, 'mask_type': 'circular', 'projection_percentage': 100.0, 'bouton_number': 50, 'receptor_location': None, 'redundancy': 100.0, 'execution_mode': 'both', 'script_format': 'both'}


In [17]:
#EI_connect.get_info()

### Connect Inh.to Exc populations using NEST connection rules

In [18]:
# Create a population 
IE_connect = SNNbuilder_Connection('connection from Inh to Exc Neurons')
    
# Configure the connection rule parameters

IE_connect.configure(
    name='IE indegree',
    connection_rule='fixed_indegree',
    connection_dict={'allow_autapses': False}, 
    synapse_model= 'static_synapse',
    synapse_dict= {'weight': 10.0, 'delay': 1.0},

    # Execution options
    execution_mode="both",  # Execute and generate script
    script_format="both",    # Python and notebook formats

    redundancy=50.0, #to adjust the indegree
    axon_radius= 1.0
    )
print(f"Rule: {IE_connect._parameters}")

Rule: {'name': 'IE indegree', 'connection_rule': 'fixed_indegree', 'connection_dict': {'allow_autapses': False}, 'synapse_model': 'static_synapse', 'synapse_dict': {'weight': 10.0, 'delay': 1.0}, 'template_suffix': '_custom', 'axon_organization': 'focused', 'axon_radius': 1.0, 'mask_type': 'circular', 'projection_percentage': 100.0, 'bouton_number': 50, 'receptor_location': None, 'redundancy': 50.0, 'execution_mode': 'both', 'script_format': 'both'}


### Add the connection nodes to the workflow

In [19]:
workflow.add_node(EI_connect)
workflow.add_node(IE_connect)

### Check available ports

In [20]:
my_pop.get_info()['output_ports'], EI_connect.get_info()['input_ports']

({'nest_population': {'type': 'object',
   'description': 'NEST population object (if execution mode)'},
  'population_data': {'type': 'dict',
   'description': 'Population metadata and configuration'},
  'python_script': {'type': 'str',
   'description': 'Generated Python code for population creation'},
  'notebook_cell': {'type': 'str',
   'description': 'Generated Jupyter notebook cell code for population creation'}},
 {'source_nest_population': {'type': 'object',
   'description': 'Source NEST population object',
   'optional': True},
  'source_population_metadata': {'type': 'dict',
   'description': 'Source population metadata and properties',
   'optional': True},
  'source_neuron_properties': {'type': 'dict',
   'description': 'Source neuron biological properties (including PSP data)',
   'optional': True},
  'target_nest_population': {'type': 'object',
   'description': 'Target NEST population object',
   'optional': True},
  'target_population_metadata': {'type': 'dict',
   'd

### Connect Population nodes with Connection rule nodes (E->I, I->E)

In [21]:
workflow.connect( 
    my_pop.name, "nest_population",
    EI_connect.name, "source_nest_population")
workflow.connect( 
    my_pop.name, "population_data",
    EI_connect.name, "source_population_metadata")

workflow.connect( 
    my_pop_inh.name, "nest_population",
    EI_connect.name, "target_nest_population")
workflow.connect( 
    my_pop_inh.name, "population_data",
    EI_connect.name, "target_population_metadata")

In [22]:
workflow.connect( 
    my_pop_inh.name, "nest_population",
    IE_connect.name, "source_nest_population")
workflow.connect( 
    my_pop_inh.name, "population_data",
    IE_connect.name, "source_population_metadata")

workflow.connect( 
    my_pop.name, "nest_population",
    IE_connect.name, "target_nest_population")
workflow.connect( 
    my_pop.name, "population_data",
    IE_connect.name, "target_population_metadata")

In [23]:
workflow.get_connection_count()

12

### Create stimulation nodes (poisson generator and DC generator)

In [24]:
#Create a stimulus
stim_pg = SNNbuilder_Stimulation("poisson_stimulation noise")

In [25]:
# defaul parameters
#stim_pg._parameters

In [26]:
# Define parameters
stim_pg.configure(
    # Stimulation type and devices
    stimulation_type= 'poisson_generator',
    number_of_devices= 1,
        
    # Poisson generator parameters
    poisson_parameters= {
        'rate': 50.0,      # Hz
        'start': 100.0,    # ms
        'stop': 900.0      # ms
    },
        
    # Target all neurons in population
    target_identification= 'full_population',
        
    # Synapse specification
    synapse_specification= {
            'synapse_model': 'static_synapse',
            'weight': 2.5,     # pA
            'delay': 1.5       # ms
        },
        
        # Template and execution options
        template_suffix= '_custom',
        script_format= 'both',        # Generate both script and notebook
        execution_mode= 'both' #  Execute and generate script
    )

print(f"Stimulus: {stim_pg._parameters}")

Stimulus: {'stimulation_type': 'poisson_generator', 'number_of_devices': 1, 'poisson_parameters': {'rate': 50.0, 'start': 100.0, 'stop': 900.0}, 'dc_parameters': {'label': 'dc_stim', 'start': 0.0, 'stop': 1000.0, 'amplitude': 100.0}, 'synapse_specification': {'synapse_model': 'static_synapse', 'weight': 2.5, 'delay': 1.5}, 'target_identification': 'full_population', 'volume_area_specification': {'center_coordinates': [0.0, 0.0, 0.0], 'radius': 0.5}, 'execution_mode': 'both', 'script_format': 'both', 'template_suffix': '_custom'}


In [27]:
#Create a stimulus
stim_dc = SNNbuilder_Stimulation("direct current stimulation")

stim_dc.configure(
    # Stimulation type and devices
    stimulation_type= 'dc_generator',
    number_of_devices= 1, #always 1 for DC generator

    #dc parameters
    dc_parameters= {
        'label': 'dc_stim',
        'start': 500.0,
        'stop': 700.0,
        'amplitude': 100.0
    },
        
    # Target all neurons in population
    target_identification= 'define_volume_area',
    volume_area_specification= {
        'center_coordinates': [0.0, 0.0, 0.0],
        'radius': 0.5
    },

    script_format= 'both',        # Generate both script and notebook
    execution_mode= 'both' #  Execute and generate script
)

print(f"Stimulus: {stim_dc._parameters}")

Stimulus: {'stimulation_type': 'dc_generator', 'number_of_devices': 1, 'poisson_parameters': {'label': 'poisson_stim', 'start': 0.0, 'stop': 1000.0, 'rate': 100.0}, 'dc_parameters': {'label': 'dc_stim', 'start': 500.0, 'stop': 700.0, 'amplitude': 100.0}, 'synapse_specification': {'synapse_model': 'static_synapse', 'weight': 1.0, 'delay': 1.0}, 'target_identification': 'define_volume_area', 'volume_area_specification': {'center_coordinates': [0.0, 0.0, 0.0], 'radius': 0.5}, 'execution_mode': 'both', 'script_format': 'both', 'template_suffix': '_custom'}


### Add stimulation nodes to the workflow

In [28]:
workflow.add_node(stim_pg)
workflow.add_node(stim_dc)

In [29]:
my_pop.get_info()['output_ports'], stim_pg.get_info()['input_ports']

({'nest_population': {'type': 'object',
   'description': 'NEST population object (if execution mode)'},
  'population_data': {'type': 'dict',
   'description': 'Population metadata and configuration'},
  'python_script': {'type': 'str',
   'description': 'Generated Python code for population creation'},
  'notebook_cell': {'type': 'str',
   'description': 'Generated Jupyter notebook cell code for population creation'}},
 {'nest_population': {'type': 'object',
   'description': 'NEST population object to stimulate',
   'optional': False},
  'population_data': {'type': 'dict',
   'description': 'Population metadata including spatial information',
   'optional': False}})

### Connect stimulation nodes to population nodes

In [30]:
workflow.connect( 
    my_pop.name, "nest_population",
    stim_pg.name, "nest_population")
workflow.connect( 
    my_pop.name, "population_data",
    stim_pg.name, "population_data")

workflow.connect( 
    my_pop.name, "nest_population",
    stim_dc.name, "nest_population")
workflow.connect( 
    my_pop.name, "population_data",
    stim_dc.name, "population_data")

In [31]:
workflow.get_connection_count()

16

### Create recordable nodes (spike recorder)

In [32]:
#Create a recordable for exc. population
sp_exc = SNNbuilder_Recordable("spike detector for Exc. pop")

sp_exc.configure(
    # recording type and devices
    recording_type= 'spike_recorder',

    #spike_recorder parameters
    spike_recorder_parameters= {
        'record_to': 'ascii',
        'start': 0.0,
        'stop': 1000.0,
    },
        
    # Target all neurons in population
    target_identification= 'full_population',

    script_format= 'both',        # Generate both script and notebook
    execution_mode= 'both' #  Execute and generate script
)

print(f"Stimulus: {sp_exc._parameters}")

Stimulus: {'recording_type': 'spike_recorder', 'number_of_devices': 1, 'spike_recorder_parameters': {'record_to': 'ascii', 'start': 0.0, 'stop': 1000.0}, 'target_identification': 'full_population', 'volume_area_specification': {'center_coordinates': [0.0, 0.0, 0.0], 'radius': 0.5}, 'script_format': 'both', 'execution_mode': 'both'}


In [33]:
#Create an other recordable
sp_inh = SNNbuilder_Recordable("spike detector for some neurons at Inh. pop")

sp_inh.configure(
    # recording type and devices
    recording_type= 'spike_recorder',

    #spike_recorder parameters
    spike_recorder_parameters= {
        'record_to': 'ascii',
        'start': 0.0,
        'stop': 1000.0,
    },
        
    # Target all neurons in population
    target_identification= 'define_volume_area',
    
    volume_area_specification= {
        'center_coordinates': [0.0, 0.0, 0.0],
        'radius': 0.5
    },

    script_format= 'both',        # Generate both script and notebook
    execution_mode= 'both' #  Execute and generate script
)

print(f"Stimulus: {sp_inh._parameters}")

Stimulus: {'recording_type': 'spike_recorder', 'number_of_devices': 1, 'spike_recorder_parameters': {'record_to': 'ascii', 'start': 0.0, 'stop': 1000.0}, 'target_identification': 'define_volume_area', 'volume_area_specification': {'center_coordinates': [0.0, 0.0, 0.0], 'radius': 0.5}, 'script_format': 'both', 'execution_mode': 'both'}


In [34]:
my_pop.get_info()['output_ports'], sp_exc.get_info()['input_ports']

({'nest_population': {'type': 'object',
   'description': 'NEST population object (if execution mode)'},
  'population_data': {'type': 'dict',
   'description': 'Population metadata and configuration'},
  'python_script': {'type': 'str',
   'description': 'Generated Python code for population creation'},
  'notebook_cell': {'type': 'str',
   'description': 'Generated Jupyter notebook cell code for population creation'}},
 {'nest_population': {'type': 'object',
   'description': 'NEST population object to record from',
   'optional': False},
  'population_data': {'type': 'dict',
   'description': 'Population metadata including spatial information',
   'optional': False}})

### Add recordable nodes to the workflow

In [35]:
workflow.add_node(sp_exc)
workflow.add_node(sp_inh)

### Connect population nodes to recordable nodes

In [36]:
workflow.connect( 
    my_pop.name, "nest_population",
    sp_exc.name, "nest_population")
workflow.connect( 
    my_pop.name, "population_data",
    sp_exc.name, "population_data")

workflow.connect( 
    my_pop.name, "nest_population",
    sp_inh.name, "nest_population")
workflow.connect( 
    my_pop.name, "population_data",
    sp_inh.name, "population_data")

In [37]:
workflow.get_connection_count()

20

### Build the workflow

In [38]:
# Build the workflow
workflow_ready = workflow.build()

# Print workflow information
print(workflow_ready)

Workflow: Simple population of spiking neurons
Nodes:
  pyramidal neuron
  Parvalbumin-expressing interneuron
  cortical_population
  inhibitory cortical population
  connection from Exc. to Inh Neurons.
  connection from Inh to Exc Neurons
  poisson_stimulation noise
  direct current stimulation
  spike detector for Exc. pop
  spike detector for some neurons at Inh. pop
Connections:
  pyramidal neuron.nest_model_name -> cortical_population.nest_model_name
  pyramidal neuron.biological_properties -> cortical_population.biological_properties
  Parvalbumin-expressing interneuron.nest_model_name -> inhibitory cortical population.nest_model_name
  Parvalbumin-expressing interneuron.biological_properties -> inhibitory cortical population.biological_properties
  cortical_population.nest_population -> connection from Exc. to Inh Neurons..source_nest_population
  cortical_population.population_data -> connection from Exc. to Inh Neurons..source_population_metadata
  inhibitory cortical populat

### Workflow Execution

In [39]:
# Execute the workflow
print("Executing workflow...")
success = workflow_ready.execute()

if success:
    print("\nWorkflow execution completed successfully!")
else:
    print("\nWorkflow execution failed!")

Executing workflow...
Executing node: Parvalbumin-expressing interneuron
[Parvalbumin-expressing interneuron] Validating neuron parameters...
[Parvalbumin-expressing interneuron] Parameter validation completed successfully
[Parvalbumin-expressing interneuron] Creating NEST neuron...
[Parvalbumin-expressing interneuron] Copying model: iaf_psc_alpha -> iaf_psc_alpha_PV_custom
[Parvalbumin-expressing interneuron] Applied inhibitory cell class defaults: {'V_th': -52.0, 'V_reset': -65.0, 'tau_m': 10.0, 't_ref': 1.0}
[Parvalbumin-expressing interneuron] Setting defaults: {'C_m': 200.0, 'V_th': -52.0, 'V_reset': -65.0, 'tau_m': 10.0, 't_ref': 1.0}
[Parvalbumin-expressing interneuron] Creating neuron from template: iaf_psc_alpha_PV_custom
[Parvalbumin-expressing interneuron] NEST neuron created successfully: NodeCollection(metadata=None, model=iaf_psc_alpha_PV_custom, size=1, first=1)
[Parvalbumin-expressing interneuron] Generating Python script...
[Parvalbumin-expressing interneuron] Python s

### Check nodes execution sequence

In [40]:
execution_sequence = workflow_ready.get_execution_sequence()
print(execution_sequence)

{'workflow_name': 'Simple population of spiking neurons', 'execution_timestamp': '2025-09-27 23:44:30', 'total_nodes': 10, 'execution_sequence': [{'node_name': 'Parvalbumin-expressing interneuron', 'node_instance': <neuroworkflow.nodes.network.SNNbuilder_SingleNeuron.SNNbuilder_SingleNeuron object at 0x176e3b770>, 'node_type': 'single_neuron_builder', 'execution_order': 0, 'timestamp': 1758984267.475772, 'duration': 0.0019462108612060547, 'success': True, 'has_python_script': True, 'has_notebook_cell': True, 'output_ports': ['nest_neuron', 'nest_model_name', 'python_script', 'notebook_cell', 'neuron_metadata', 'biological_properties', 'nest_properties']}, {'node_name': 'inhibitory cortical population', 'node_instance': <neuroworkflow.nodes.network.SNNbuilder_Population.SNNbuilder_Population object at 0x11f8db1d0>, 'node_type': 'population_builder', 'execution_order': 1, 'timestamp': 1758984267.477727, 'duration': 0.0036420822143554688, 'success': True, 'has_python_script': True, 'has_n

### Import SNNbuilder function from utils folder to generate code for simulation 

In [41]:
from neuroworkflow.utils import export_workflow_scripts

### generate code (.py and .ipynb)

In [42]:
files = export_workflow_scripts(
        execution_sequence=execution_sequence,
        output_dir="./",           # Custom directory
        export_python=True,                              # Export Python script
        export_notebook=True,                            # Export Jupyter notebook
        filename_base="code_gen_test3",                   # Custom filename base
        deduplicate_imports=True,                        # Remove duplicate imports
        add_metadata=True                                # Add workflow metadata
)

Processing workflow: Simple population of spiking neurons
Total nodes in execution order: 10
Processing node: Parvalbumin-expressing interneuron (has_python: True, has_notebook: True)
  → Collected Python script (1077 chars)
  → Collected notebook cell (1417 chars) + python script (1077 chars)
Processing node: inhibitory cortical population (has_python: True, has_notebook: True)
  → Collected Python script (1388 chars)
  → Collected notebook cell (377 chars) + python script (1388 chars)
Processing node: pyramidal neuron (has_python: True, has_notebook: True)
  → Collected Python script (1042 chars)
  → Collected notebook cell (1468 chars) + python script (1042 chars)
Processing node: cortical_population (has_python: True, has_notebook: True)
  → Collected Python script (1566 chars)
  → Collected notebook cell (390 chars) + python script (1566 chars)
Processing node: spike detector for some neurons at Inh. pop (has_python: True, has_notebook: True)
  → Collected Python script (1639 char

### other useful commands

In [43]:
EI_connect.get_info()['output_ports'], EI_connect.get_output_port('connection_summary').value

({'nest_connections': {'type': 'object',
   'description': 'Created NEST connection object'},
  'custom_synapse_model': {'type': 'str',
   'description': 'Name of the created custom synapse model'},
  'connection_summary': {'type': 'dict',
   'description': 'Summary of created connections and parameters'},
  'python_script': {'type': 'str',
   'description': 'Generated Python script for connection creation'},
  'notebook_cell': {'type': 'str',
   'description': 'Generated Jupyter notebook cell'}},
 {'source_population_size': 800,
  'target_population_size': 100,
  'total_connections': 400,
  'connection_density': 0.005,
  'connection_rule': 'fixed_indegree',
  'synapse_model': 'static_synapse_custom_connection_from_Exc._to_Inh_Neurons.',
  'creation_timestamp': '2025-09-27 23:11:46'})

In [44]:
print(my_neuron.get_output_port("python_script").value)

# Single Neuron: Pyramidal Neuron L2/3 (PyL23)
# Generated by NeuroWorkflow SingleNeuronBuilderNode
# Model Type: point_process
# Cell Class: excitatory

import nest

# === BIOLOGICAL PROPERTIES ===
# Dendrite extent: 300.0 μm
# Dendrite diameter: 2.5 μm
# Neurotransmitters: ['AMPA', 'NMDA']
# PSP amplitudes: {'AMPA': 0.8, 'NMDA': 0.4}
# Rise times: {'AMPA': 2.0, 'NMDA': 15.0}
# Firing rates - Resting: [2.0, 8.0] Hz
# Firing rates - Active: [15.0, 40.0] Hz
# Firing rates - Maximum: [60.0, 120.0] Hz

# === NEST MODEL CREATION ===
nest.CopyModel("iaf_psc_alpha", "iaf_psc_alpha_PyL23_custom")
nest.SetDefaults("iaf_psc_alpha_PyL23_custom", {
                        "V_th": -50.0,
                        "C_m": 200.0,
                        "tau_m": 15.0,
                        "V_reset": -70.0,
                        "t_ref": 2.0,
                        "I_e": 200.0
                    })
pyl23 = nest.Create("iaf_psc_alpha_PyL23_custom", 1)

print(f"Created neuron: {pyl23}")
print(f"Mo

In [45]:
print(my_neuron.get_output_port("notebook_cell").value)

# Markdown Cell
```markdown
## Neuron: Pyramidal Neuron L2/3 (PyL23)

**Biological Properties:**
- Model Type: point_process
- Cell Class: excitatory
- Dendrite Extent: 300.0 μm
- Dendrite Diameter: 2.5 μm
- Neurotransmitters: AMPA, NMDA

**NEST Model:**
- Base Model: iaf_psc_alpha
- Custom Parameters: {'V_th': -50.0, 'C_m': 200.0, 'tau_m': 15.0, 'V_reset': -70.0, 't_ref': 2.0, 'I_e': 200.0}
```

# Code Cell
```python
# Single Neuron: Pyramidal Neuron L2/3 (PyL23)
# Generated by NeuroWorkflow SingleNeuronBuilderNode
# Model Type: point_process
# Cell Class: excitatory

import nest

# === BIOLOGICAL PROPERTIES ===
# Dendrite extent: 300.0 μm
# Dendrite diameter: 2.5 μm
# Neurotransmitters: ['AMPA', 'NMDA']
# PSP amplitudes: {'AMPA': 0.8, 'NMDA': 0.4}
# Rise times: {'AMPA': 2.0, 'NMDA': 15.0}
# Firing rates - Resting: [2.0, 8.0] Hz
# Firing rates - Active: [15.0, 40.0] Hz
# Firing rates - Maximum: [60.0, 120.0] Hz

# === NEST MODEL CREATION ===
nest.CopyModel("iaf_psc_alpha", "iaf_psc_al

In [46]:
print(my_pop.get_output_port("python_script").value)

# Population: Layer 2/3 Pyramidal Neurons (L23Pyr)
# Generated by NeuroWorkflow PopulationBuilderNode
# Brain Region: Primary Visual Cortex
# Population Size: 800 neurons (single hemisphere)
# Model Type: point_process
# Spatial: 3D

import numpy as np

# === POPULATION BIOLOGICAL CONTEXT ===
# Density: 25000 neurons/mm³
# Tissue volume: 0.2 mm³
# Reference: Schüz & Palm (1989)
# Species: human
# Estimation method: stereological counting
# Spatial bounds: {'x': (0.0, 2.0), 'y': (0.0, 2.0), 'z': (0.0, 1.0)}

# === SPATIAL POSITIONS ===
# 3D positions for 800 neurons
# Spatial bounds: {'x': (0.0, 2.0), 'y': (0.0, 2.0), 'z': (0.0, 1.0)}

# Generate spatial positions
positions = np.zeros((800, 3))
positions[:, 0] = np.random.uniform(0.0, 2.0, 800)  # x coordinates
positions[:, 1] = np.random.uniform(0.0, 2.0, 800)  # y coordinates
positions[:, 2] = np.random.uniform(0.0, 1.0, 800)  # z coordinates

# === POPULATION CREATION ===
# Using neuron model from SingleNeuronBuilderNode: iaf_psc_alp

In [47]:
print(my_pop.get_output_port("notebook_cell").value)

# Population: Layer 2/3 Pyramidal Neurons
# 800 neurons, 3D

# Create spatial positions
positions = np.random.uniform([0.0, 0.0, 0.0], [2.0, 2.0, 1.0], (800, 3))
nest_positions = nest.spatial.free(positions.tolist())
l23pyr = nest.Create('iaf_psc_alpha_PyL23_custom', 800, positions=nest_positions)
neuron_status = nest.GetStatus(l23pyr[0])
print(f"Status of first neuron: {neuron_status}")


In [48]:
my_pop.get_input_port("biological_properties").get_value()

{'identification': {'name': 'Pyramidal Neuron L2/3',
  'acronym': 'PyL23',
  'model_type': 'point_process',
  'cell_class': 'excitatory'},
 'signaling': {'neurotransmitter_types': ['AMPA', 'NMDA'],
  'psp_amplitudes': {'AMPA': 0.8, 'NMDA': 0.4},
  'rise_times': {'AMPA': 2.0, 'NMDA': 15.0}},
 'morphology': {'dendrite_extent_um': 300.0, 'dendrite_diameter_um': 2.5},
 'activity': {'firing_rate_resting_hz': [2.0, 8.0],
  'firing_rate_active_hz': [15.0, 40.0],
  'firing_rate_maximum_hz': [60.0, 120.0],
  'firing_rate_disease_hz': [0.1, 2.0]}}

In [49]:
print(f"Population input ports: {my_pop.get_info()['output_ports'].keys()}")    

Population input ports: dict_keys(['nest_population', 'population_data', 'python_script', 'notebook_cell'])


In [50]:
my_pop.get_output_port("population_data").value

{'population_size': 800,
 'name': 'Layer 2/3 Pyramidal Neurons',
 'acronym': 'L23Pyr',
 'model_name': 'iaf_psc_alpha_PyL23_custom',
 'positions': array([[0.35009033, 0.92161928, 0.87127944],
        [1.04853434, 0.58767353, 0.02763789],
        [0.02977129, 1.61820591, 0.23041876],
        ...,
        [0.51034784, 1.65810333, 0.08298278],
        [0.39637357, 1.86233676, 0.72733311],
        [1.38457923, 1.98241354, 0.09834153]]),
 'spatial_dimensions': '3D',
 'biological_properties': {'identification': {'name': 'Pyramidal Neuron L2/3',
   'acronym': 'PyL23',
   'model_type': 'point_process',
   'cell_class': 'excitatory'},
  'signaling': {'neurotransmitter_types': ['AMPA', 'NMDA'],
   'psp_amplitudes': {'AMPA': 0.8, 'NMDA': 0.4},
   'rise_times': {'AMPA': 2.0, 'NMDA': 15.0}},
  'morphology': {'dendrite_extent_um': 300.0, 'dendrite_diameter_um': 2.5},
  'activity': {'firing_rate_resting_hz': [2.0, 8.0],
   'firing_rate_active_hz': [15.0, 40.0],
   'firing_rate_maximum_hz': [60.0, 120.

In [51]:
biological_properties = my_pop.get_output_port("population_data").value['biological_properties']['identification']['cell_class']
print(biological_properties)

excitatory


In [45]:
EI_connect.get_info()['output_ports']

{'nest_connections': {'type': 'object',
  'description': 'Created NEST connection object'},
 'custom_synapse_model': {'type': 'str',
  'description': 'Name of the created custom synapse model'},
 'connection_summary': {'type': 'dict',
  'description': 'Summary of created connections and parameters'},
 'python_script': {'type': 'str',
  'description': 'Generated Python script for connection creation'},
 'notebook_cell': {'type': 'str',
  'description': 'Generated Jupyter notebook cell'}}